In [1]:
import os

import configparser
import pandas as pd

from sqlalchemy import create_engine, text

### Database taken from https://github.com/classicdb/database.
### Firstly you need to create new user for MYSQL.
### Secondly you need to create new empy database and grant all privileges to the user.
### Thirdly run command below to fill your database with data. The script will ask username, password and database name. They will be written in config file `main.ini`.
```bash
python3 ./fill_db.py
```

In [2]:
# create config file if not exist
if 'main.ini' not in os.listdir():
    config = configparser.ConfigParser()
    print('Type username, password and database name of your mysql separated whitespace')
    username, password, database = input().split(' ')
    
    config['SQL'] = {'username': username, 'password': password, 'database': database}
    with open('main.ini', 'w') as configfile:
        config.write(configfile)

In [3]:
# read config file
config = configparser.ConfigParser()
config.read('main.ini')

['main.ini']

In [4]:
engine = create_engine('mysql+pymysql://{}:{}@localhost/{}'.format(config['SQL']['username'],
                                                                   config['SQL']['password'],
                                                                   config['SQL']['database']))

In [5]:
# all tables in database
columns = pd.read_sql_query("""
SHOW tables;
""", engine)
columns['Tables_in_mongos'].to_list()

['areatrigger_involvedrelation',
 'areatrigger_tavern',
 'areatrigger_teleport',
 'battleground_events',
 'battleground_template',
 'battlemaster_entry',
 'command',
 'conditions',
 'creature',
 'creature_addon',
 'creature_ai_scripts',
 'creature_ai_summons',
 'creature_ai_texts',
 'creature_battleground',
 'creature_equip_template',
 'creature_equip_template_raw',
 'creature_involvedrelation',
 'creature_linking',
 'creature_linking_template',
 'creature_loot_template',
 'creature_model_info',
 'creature_movement',
 'creature_movement_template',
 'creature_onkill_reputation',
 'creature_questrelation',
 'creature_template',
 'creature_template_addon',
 'creature_template_classlevelstats',
 'creature_template_spells',
 'custom_texts',
 'db_script_string',
 'db_version',
 'dbscripts_on_creature_death',
 'dbscripts_on_creature_movement',
 'dbscripts_on_event',
 'dbscripts_on_go_template_use',
 'dbscripts_on_go_use',
 'dbscripts_on_gossip',
 'dbscripts_on_quest_end',
 'dbscripts_on_quest

In [6]:
items = pd.read_sql_query("""
SELECT 
  name, AllowableClass, InventoryType, subclass, Quality, bonding, 
  armor, holy_res, fire_res, nature_res, frost_res, shadow_res, arcane_res,
  stat_type1, stat_value1, 
  stat_type2, stat_value2, 
  stat_type3, stat_value3, 
  stat_type4, stat_value4,
  dmg_min1, dmg_max1, dmg_type1,
  dmg_min2, dmg_max2, dmg_type2,
  dmg_min3, dmg_max3, dmg_type3,
  delay,
  spelltrigger_1, s1.SpellName AS sp1, s1.EffectBasePoints1 AS spb1, 
  spelltrigger_2, s2.SpellName AS sp2, s2.EffectBasePoints1 AS spb2, 
  spelltrigger_3, s3.SpellName AS sp3, s3.EffectBasePoints1 AS spb3
FROM item_template
  LEFT JOIN spell_template as s1 ON spellid_1 = s1.Id
  LEFT JOIN spell_template as s2 ON spellid_2 = s2.Id
  LEFT JOIN spell_template as s3 ON spellid_3 = s3.Id
""", engine)

In [7]:
item_class_map = {'consumable': 0, 'container': 1, 'weapon': 2, 'armor': 4, 'reagent': 5, 'projectile': 6,
                  'trade good': 7, 'recipe': 9, 'quiver': 11, 'quest': 12, 'key': 13, 'miscellaneous': 15}

item_subclass_map = {'cloth': 1, 'leather': 2, 'mail': 3, 'plate': 4}

item_quality_map = {'poor': 0, 'common': 1, 'uncommon': 2, 'rare': 3, 'epic': 4, 'legendary': 5}

item_type_map = {'head': [1], 'neck': [2], 'shoulders': [3], 'chest': [4, 5, 20],
                 'waist': [6], 'legs': [7], 'feet': [8], 'wrists': [9], 
                 'hands': [10], 'finger': [11], 'trinket': [12], 
                 'one-hand': [13, 21], 'shield': [14], 'ranged': [15], 
                 'back': [16], 'two_hand': [17], 'offhand': [22], 'thrown': [25],
                 'gun': [26], 'bow': [15], 'left-hand': [22], 'relic': 28}

stat_reverse_map = {1: 'health', 3: 'agility', 4: 'strenght', 5: 'intellect', 6: 'spirit', 7: 'stamina'}

stat_map = {'health': 1, 'agility': 3, 'strenght': 4, 'intellect': 5, 'spirit': 6, 'stamina': 7}

spell_map = {'on use': 0, 'on equip': 1, 'chance on hit': 2, 'soulstone': 4, 'on use without delay': 5}

bounding_map = {'no binding': 0, 'bind on pickup': 1, 'bind on equip': 2, 'bind on use': 3 ,'quest item': 4}

damage_map = {'physical': 0, 'holy': 1, 'fire': 2, 'nature': 3, 'frost': 4, 'shadow': 5, 'arcane': 6}

race_map = {'human': 1, 'orc': 2, 'dwarf': 3, 'elf': 4, 
            'undead': 5, 'tauren': 6, 'gnome': 7, 'troll': 8}

class_map = {'warrior': 1, 'paladin': 2, 'hunter': 3, 'rogue': 4, 'prist': 5, 
             'shaman': 7, 'mage': 8, 'warlock': 9, 'druid': 11}

In [8]:
items.loc[(items['InventoryType'].isin(item_type_map['chest'])) 
          & (items['subclass'] == item_subclass_map['cloth'])
          & (items['Quality'] == item_quality_map['epic']),].head(5).T

,216,7357,7898,8297,8298
name,Robes of Insight,Embrace of the Wind Serpent,Alanna's Embrace,Robe of the Archmage,Robe of the Void
AllowableClass,-1,-1,-1,31360,31488
InventoryType,20,20,20,20,20
subclass,1,1,1,1,1
Quality,4,4,4,4,4
bonding,2,1,1,1,1
armor,74,86,96,96,96
holy_res,0,0,0,0,0
fire_res,0,0,0,0,0
nature_res,0,12,0,0,0


In [9]:
spells = pd.read_sql_query("""
SELECT *
FROM playercreateinfo_spell;
""", engine)
spells.head()

,race,class,Spell,Note
0,1,1,78,Heroic Strike
1,1,1,81,Dodge
2,1,1,107,Block
3,1,1,196,One-Handed Axes
4,1,1,198,One-Handed Maces


In [10]:
spell_bonus = pd.read_sql_query("""
SELECT *
FROM spell_bonus_data
WHERE comments like %(like)s
""", engine, params={'like': '%Warlock%'})
spell_bonus.head()

,entry,direct_bonus,dot_bonus,ap_bonus,ap_dot_bonus,comments
0,172,0.0000,0.1560,0.0,0.0,Warlock - Corruption
1,348,0.2000,0.2000,0.0,0.0,Warlock - Immolate
2,603,0.0000,2.0000,0.0,0.0,Warlock - Curse of Doom
3,686,0.8571,0.0000,0.0,0.0,Warlock - Shadow Bolt
4,755,0.0000,0.4485,0.0,0.0,Warlock - Health Funnel


In [11]:
# base hp and mana
base_hp_mana = pd.read_sql_query("""
SELECT *
FROM player_classlevelstats;
""", engine)
base_hp_mana.head()

,class,level,basehp,basemana
0,1,1,20,0
1,1,2,29,0
2,1,3,38,0
3,1,4,47,0
4,1,5,56,0


In [12]:
# base stats
stats = pd.read_sql_query("""
SELECT *
FROM player_levelstats;
""", engine)

In [13]:
stats.head()

,race,class,level,str,agi,sta,inte,spi
0,1,1,1,23,20,22,20,21
1,1,1,2,24,21,23,20,21
2,1,1,3,25,21,24,20,22
3,1,1,4,26,22,25,20,22
4,1,1,5,28,23,26,20,22
